# Preparation Our Data Pipeline
This notebook will guide you through how to create your ETL pipeline

### 1. Import necssary libraries and import our dataset.
- Import Python libraries
- Load `disaster_messages.csv` into a dataframe (df) and explore the first 5 rows
- Load `disaster_categories.csv` into a dataframe (df) and  explore the first 5 rows.

In [2]:
# Import necessary libraries
## these two lib for processing and transform our dataset
import numpy as np
import pandas as pd
## For visualization
import matplotlib.pyplot as plt
## import this for using SQL language
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine


In [8]:
# Get our dataset of  disaster_messages.csv 
df_mess = pd.read_csv("data/disaster_messages.csv")
#Check result
df_mess.head(n=5)

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [11]:
# Get our dataset of  disastercategories.csv 
df_cat = pd.read_csv("data/disaster_categories.csv ")
# check result
df_cat.head(n=5)

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


Looking at the `df_cat`, we can see the issue related in this dataset is the `;` between data. Therfore, we will need to split this into seperate columns for later transformation

### 2. Combine datasets.
We are now need to combine two dataset together into master dataframe, named `df`. We will take these two steps for combination
- Combine two dataset with `Merge` command using the common id between the `df_mess` and `df_cat` datasets 
- Assign this merged dataset together into `df` dataframe, which will be transformed and cleaned later in this notebook

In [12]:
# Do the merge command 
## using outer merge to get everything from both dataset based on their common "id"
df = df_mess.merge(df_cat, how ='outer', on =['id'])
# check the result
df.head(n=5)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Break `df_cat` into separate category columns.
This dataset cointains `;`, which create diffulcities for our ETL pipeline to classifiy and categorize so we would need to take approriate action with `split` command. Let's do the split:
- Split the values in the `df_cat` column on the `;` character so that each value becomes a separate column. During looking the answers to resolve this issue, I found [this method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html) very detail, and helpful with examples for your reference! Ensure you also enable `expand=True` when doing the split.


In [13]:
# create a new columns for our df_cat dataframe 
df_cat = df['categories'].str.split(';', expand=True)
# Check my result
df_cat.head(n=5)
# we now have new 36 individual category columns, which has been loaded back into df_cat dataframe

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


However, we will encounter new small issudes related to naming the headers for our new columns. Let's just name our new columns based on first row of categories in the dataframe for our header:
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `df_cat` with new column names.

In [20]:
# Get the first row (n=1) of the categories dataframe
row = df_cat.head(n=1)

# use this row to retrieve a list of new headers name for our df_cat.
## Instead of using the define def function, let's use the lambda function for slicing each string up until the last two characters.
cat_headers = row.applymap(lambda x: x[:-2]).iloc[0,:]
print(cat_headers)

0
related                                  related
request                                  request
offer                                      offer
aid_related                          aid_related
medical_help                        medical_help
medical_products                medical_products
search_and_rescue              search_and_rescue
security                                security
military                                military
child_alone                          child_alone
water                                      water
food                                        food
shelter                                  shelter
clothing                                clothing
money                                      money
missing_people                    missing_people
refugees                                refugees
death                                      death
other_aid                              other_aid
infrastructure_related    infrastructure_related
transport         

In [22]:
# Change the name of old columns from df_cat into new header names with our latest and updated cat_headers
df_cat.columns = cat_headers
df_cat.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


However, looking through briefly the dataset in df_cat, they still have a lot of string in dataset such as related-1 or related-0 which could create difficulties for transforming and building accuracy ML models. Let's transform it into value such as  `0` or `1` or `2` only.

In [23]:
# Count the quantitave of value present in ourdataset
df_cat.related.value_counts()

related-1    20042
related-0     6140
related-2      204
Name: related, dtype: int64

### 4. Convert category values to just numbers 0, 1, 2 or other numbers only.
Let's do convert our category values to numbers only 
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`, or `related-2` becomes `2`. Followed the conversion to number, we also need to Convert the string (datatype) to a numeric value (datatype).
- For the transformation and conversion, we can apply `astype(str)` for transformation from string to numeric value. For the conversion from `related-0` to `0`, we could apply the function [indexing with string](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html) like indexing, by including `.str` after the Series


In [24]:
# Use the for loop to loop through columns in df_cat
## You can copy each statement and followed with .head() to check the result
for column in df_cat:
    # Assign value of the column = the last character in the column. For example related-1 = 1
    df_cat[column] = df_cat[column].astype(str).str[-1]
    
    # convert column from string to numeric
    df_cat[column] = df_cat[column].astype(int)
# Check the result
df_cat.head(n=5)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Re-check again the result
df_cat.related.value_counts()

1    20042
0     6140
2      204
Name: related, dtype: int64

However, after checking the meaning of 0,1, and 2, we need to replace to 2 as well. Therefore, I will replace it with value 1

In [29]:
# To replacement
df_cat['related'] = df_cat['related'].replace(to_replace=2, value=1)
#check the result
df_cat.related.value_counts()

1    20246
0     6140
Name: related, dtype: int64

### 5. Replace `categories` column in `df` master dataframe with our new updated category columns.
- Drop the categories column from the df dataframe due to we just have our new header names.
- Concatenate df and df_cat dataframes together.

In [30]:
# drop the old categories column from df dataframe master
## Do the drop, enable inplace=True for confirmation
df.drop('categories', axis=1, inplace=True)
# Check the result now
df.head(n=5)

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [31]:
#  replace the old old with new header names
# Let's do the concatenate the original dataframe with the new header name from df_cat
df = pd.concat([df, df_cat], axis=1)
#check the result
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
This is also important step to ensure we do not have duplicates value:
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [34]:
# Step1: check number of duplicates
df.duplicated().sum()
# Do the drop with inplace =True to confirm
df.drop_duplicates(inplace=True)

In [35]:
# Re-check number of duplicates to ensure we have Zero duplicate value
df.duplicated().sum()

0

In [39]:
# Let's do the confirmation to ensure df (master dataframe) is cleaned before moving it into save place
# remove 'id','message','original','genre' for checking with df_test dataframe
df_test = df.drop(['id','message','original','genre'], axis=1)

In [40]:
# Get it into cat_counts dataframe to check
cat_counts = df_test.sum(axis=0)
# Check the result
cat_counts

related                   20093
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

### 7. Save the clean df dataframe and its dataset into an SQLite database.
From the documentation from Pandas's documentation about transferring into SQL database, we can do this with [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) with the SQLAlchemy library.
#### Important
Remember import SQLAlchemy's `create_engine` in the first cell of this notebook to use

In [29]:
# Call the engine to import and transfer our clean dataset
engine = create_engine('sqlite:///InsertDatabaseName.db')
#enable if_exists to replace the existing dataset
df.to_sql('disaster_messages', engine, index=False, if_exists='replace')